In [51]:
"""
2751. Robot Collisions


There are n 1-indexed robots, each having a position on a line, health, and movement direction.

You are given 0-indexed integer arrays positions, healths, and a string directions (directions[i] is either 'L' for left or 'R' for right). All integers in positions are unique.

All robots start moving on the line simultaneously at the same speed in their given directions. If two robots ever share the same position while moving, they will collide.

If two robots collide, the robot with lower health is removed from the line, and the health of the other robot decreases by one. The surviving robot continues in the same direction it was going. If both robots have the same health, they are both removed from the line.

Your task is to determine the health of the robots that survive the collisions, in the same order that the robots were given, i.e. final health of robot 1 (if survived), final health of robot 2 (if survived), and so on. If there are no survivors, return an empty array.

Return an array containing the health of the remaining robots (in the order they were given in the input), after no further collisions can occur.

Note: The positions may be unsorted.

 
 

Example 1:



Input: positions = [5,4,3,2,1], healths = [2,17,9,15,10], directions = "RRRRR"
Output: [2,17,9,15,10]
Explanation: No collision occurs in this example, since all robots are moving in the same direction. So, the health of the robots in order from the first robot is returned, [2, 17, 9, 15, 10].
Example 2:



Input: positions = [3,5,2,6], healths = [10,10,15,12], directions = "RLRL"
Output: [14]
Explanation: There are 2 collisions in this example. Firstly, robot 1 and robot 2 will collide, and since both have the same health, they will be removed from the line. Next, robot 3 and robot 4 will collide and since robot 4's health is smaller, it gets removed, and robot 3's health becomes 15 - 1 = 14. Only robot 3 remains, so we return [14].
Example 3:



Input: positions = [1,2,5,6], healths = [10,10,11,11], directions = "RLRL"
Output: []
Explanation: Robot 1 and robot 2 will collide and since both have the same health, they are both removed. Robot 3 and 4 will collide and since both have the same health, they are both removed. So, we return an empty array, [].
 

Constraints:

1 <= positions.length == healths.length == directions.length == n <= 105
1 <= positions[i], healths[i] <= 109
directions[i] == 'L' or directions[i] == 'R'
All values in positions are distinct


"""

In [ ]:
"""

시뮬레이션 방식으로 각각의 로봇이 1칸 씩 이동할 때마다 충돌이 발생하는지 체크하도록 구현했다.

-> 이렇게 하면 특정 테스트 케이스에 대해서 시간 초과가 발생한다.

"""



import re
from collections import defaultdict
from typing import List
import time
from collections import Counter


class Solution:
    def survivedRobotsHealths(self, positions: List[int], healths: List[int], directions: str) -> List[int]:
        
        # 로봇 2개가 충돌하면 체력 낮은 애가 소멸, 높은 애는 health가 1만큼 깎임
        # 로봇 2개의 체력이 서로 같으면 둘 다 소멸함.

        # 한 라운드를 진행시킨 배열에서 충돌에 관련된 처리를 하면 되겠다.
        # 몇 라운드까지 갈 수 있지??
            # 임의의 라운드 A에서 검사할 때, LLLL....LLLRRRR...RRRRR 같은
            # 형식으로 배치되어 있으면 충돌이 발생하지 않는다.
            # 즉, L + R + L*R*
        
        N = len(positions)
        _directions = list(directions)
        
        self.acc_is_no_more_collision = 0
        self.acc_collide = 0
        
        pos_dict = defaultdict(list)
        collision_items = list()
        
        def init():
            for i in range(N):
                pos_dict[positions[i]].append(i)
        
        def is_no_more_collision() -> bool:
            ss = time.time()
            
            _d = list(sorted(zip(positions, _directions), key=lambda x: x[0]))
            _d = [d[1] for d in _d if d[1] is not None]

            if len(_d) == 0:
                return True
            
            pattern = r"L|R|(L*R*)"
            result = re.fullmatch(pattern, "".join(_d))
            
            ee = time.time()
            self.acc_is_no_more_collision += ee - ss
            if result:
                return True
            else:
                return False

        def fight(i, j):
            if positions[i] == positions[j]:
                winner = None
                if healths[i] < healths[j]:
                    remove_robot(i)
                    winner = j
                elif healths[i] == healths[j]:
                    remove_robot(i)
                    remove_robot(j)
                else:
                    remove_robot(j)
                    winner = i
                
                if winner is not None:
                    healths[winner] -= 1
        
        def remove_robot(i: int):
            pos_dict[positions[i]].remove(i)
            positions[i] = -1
            healths[i] = -1
            _directions[i] = None

        def removed_pos():
            return -1

        def move_L():
            for i in range(N):
                if positions[i] != removed_pos() and _directions[i] == "L":
                    pos_dict[positions[i]].remove(i)
                    positions[i] -= 1
                    pos_dict[positions[i]].append(i)
                    
                    collision_happen = len(pos_dict[positions[i]]) >= 2
                    if collision_happen:
                        collision_items.append(pos_dict[positions[i]])
        
        def move_R():
            for i in range(N):
                if positions[i] != removed_pos() and _directions[i] == "R":
                    pos_dict[positions[i]].remove(i)
                    positions[i] += 1
                    pos_dict[positions[i]].append(i)
                    
                    collision_happen = len(pos_dict[positions[i]]) >= 2
                    if collision_happen:
                        collision_items.append(pos_dict[positions[i]])

        def collide():
            collisions = defaultdict(list)
            
            ss = time.time()
            postions_counter = Counter(positions)
            for i in range(N):
                if postions_counter[positions[i]] >= 2:
                    collisions[positions[i]].append(i)
            ee = time.time()
            self.acc_collide += ee- ss
            
            if removed_pos() in collisions:
                del collisions[removed_pos()]

            
            for pos in collisions:
                i = collisions[pos][0]
                j = collisions[pos][1]
                
                fight(i, j)
                
            
        def do_one_round():
            move_L()
            collide()
            move_R()
            collide()
    

        init()
        do_one_round()
        while not is_no_more_collision():    
            do_one_round()
        
        return [h for h in healths if h != -1]


In [ ]:
"""

시간 초과에 대응하기 위해, hint를 보고 새롭게 다시 풀었다.

stack을 활용해서 마치 여는 괄호와 닫는 괄호를 체크하듯이 수행하였다.
진행 방향이 "R"인 것을 만나면 stack에 집어 넣고 "L"인 것을 만나면 stack에서 top을 뽑아서 충돌을 체크했다.

"1칸 씩 이동하는 것을 몇 라운드까지 진행해야 더 이상 충돌이 발생하지 않는가?" 라는 식의 질문이었다면 이렇게 풀 수는 없었을 것이다. 

"""



from typing import List


class Robot:
    def __init__(self, idx: int, pos: int, health: int, direction: str):
        self.idx = idx
        self.pos = pos
        self.health = health
        self.d = direction  # "R" or "L"

    def __lt__(self, other):
        # 정렬 키 값
        return self.pos < other.pos


class Solution:
    def survivedRobotsHealths(self, positions: List[int], healths: List[int], directions: str) -> List[int]:
        
        # position 기준 정렬 및 stack을 사용해서 풀 수 있다는 hint를 보았다.

        N = len(positions)
        robots = []
        survivors = []
        s = []  # stack


        def get_winner(r1: Robot, r2: Robot) -> Robot:
            if r1.health == r2.health:
                return None  # 비겼다.
            elif r1.health > r2.health:
                return r1
            else:
                return r2


        for i in range(N):
            robots.append(Robot(
                i,
                positions[i],
                healths[i],
                directions[i]
            ))

        robots = sorted(robots)


        for robot in robots:
            if robot.d == "R":
                s.append(robot)
            else:
                while True:
                    if s:
                        top = s.pop()
                        winner = get_winner(top, robot)
                        if winner is None:  # 무승부
                            break
                        elif winner == top:
                            top.health -= 1
                            s.append(top)
                            break
                        else:
                            robot.health -= 1
                    else:
                        survivors.append(robot)
                        break
        
        survivors.extend(s)

        survivors = sorted(survivors, key=lambda x: x.idx)

        return [s.health for s in survivors]
